## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')
print(len(images))
# Step through the list and search for chessboard corners
gray = []
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
print (gray.shape)       
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print (ret, mtx, dist, rvecs, tvecs)
#img = cv2.imread('test_images/test1.jpg')

#dst = cv2.undistort(img, mtx, dist, None, mtx)


#cv2.imshow('img',img)
#cv2.imshow('img2',dst)
#cv2.waitKey()


20


(720, 1280)


1.1868973603423751 [[  1.15396093e+03   0.00000000e+00   6.69705357e+02]
 [  0.00000000e+00   1.14802496e+03   3.85656234e+02]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]] [[ -2.41017956e-01  -5.30721171e-02  -1.15810354e-03  -1.28318858e-04
    2.67125300e-02]] [array([[-0.19458111],
       [-0.76190721],
       [ 0.12161603]]), array([[ 0.17850283],
       [-0.06187035],
       [ 0.00114955]]), array([[ 0.05531879],
       [-0.51892709],
       [-0.00398629]]), array([[-0.02434082],
       [-0.48814782],
       [ 0.0205404 ]]), array([[-0.33084415],
       [ 0.65914967],
       [-0.41537657]]), array([[ 0.02790357],
       [-0.70679126],
       [-0.01797426]]), array([[ 0.04125139],
       [-0.46371786],
       [-0.05619974]]), array([[ 0.0178336 ],
       [ 0.63674357],
       [ 0.00826376]]), array([[-0.44823487],
       [-0.06589252],
       [-0.01933811]]), array([[ 0.50716545],
       [-0.22122219],
       [ 0.02988717]]), array([[-0.02563264],
       [ 0.38170173],
  

## And so on and so forth...

In [4]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    #plt.imshow(img)
    #plt.waitforbuttonpress()
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(img):
    
    #printing out some stats and plotting
    print('This image is:', type(img), 'with dimensions:', img.shape)
    # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')
    image = cv2.undistort(img, mtx, dist, None, mtx)

    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    thresh = (90, 255)
    s_binary = np.zeros_like(S)
    s_binary[(S > thresh[0]) & (S <= thresh[1])] = 1
    
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)    
    thresh_min = 20
    thresh_max = 100
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1,0, ksize = 3) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    #sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    # Calculate the gradient magnitude
    #gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    #scale_factor = np.max(gradmag)/255 
    #gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    #sgrad_binary = np.zeros_like(gradmag)
    
    #sgrad_binary[(gradmag >= thresh_min) & (gradmag <= thresh_max)] = 1
    
    
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1


    #mask your edges and 
    imshape = combined_binary.shape
    vertices = np.array([[(10,imshape[0]),(imshape[1]/2, imshape[0]*0.58),
                          (imshape[1]/2, imshape[0]*0.58), (imshape[1]-10,imshape[0])]], dtype=np.int32)

    masked_image = region_of_interest(combined_binary, vertices) 
    plt.imshow(masked_image)
    plt.title('masked_image')
    plt.waitforbuttonpress()
    
    print('region of interest', vertices, type(masked_image), masked_image.shape)
    masked_image = np.expand_dims(masked_image, axis=0)
    outImage = weighted_img(masked_image, image)
    plt.imshow(outImage)
    plt.title('outImage')
    plt.waitforbuttonpress()
    '''
    #Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 1# distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 14     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 5 #minimum number of pixels making up a line
    max_line_gap = 180
    # maximum gap in pixels between connectable line segments

    line_image = hough_lines (masked_image, rho, theta, threshold, min_line_length, max_line_gap)

    #plt.imshow(line_image)

    outImage = weighted_img(line_image, image)
    return outImage
'''
img = cv2.imread('test_images/test1.jpg')
process_image(img)

This image is: <class 'numpy.ndarray'> with dimensions: (720, 1280, 3)


/home/soumya/.conda/envs/carnd-term1/lib/python3.5/site-packages/matplotlib/backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


region of interest [[[  10  720]
  [ 640  417]
  [ 640  417]
  [1270  720]]] <class 'numpy.ndarray'> (720, 1280)


error: /home/travis/miniconda/conda-bld/conda_1486587071158/work/opencv-3.1.0/modules/core/src/arithm.cpp:639: error: (-209) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function arithm_op


In [3]:

'''white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)
'''

'white_output = \'test_videos_output/solidWhiteRight.mp4\'\nclip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")\nwhite_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!\n%time white_clip.write_videofile(white_output, audio=False)\n'